## 모델 고도화
- Few-Shot Prompting
- Chain-of-Thought Prompting

In [1]:
import math
import re

from tqdm.notebook import tqdm

from eval import get_eval_data, pointwise_eval
from utils import summarize, get_train_data

In [2]:
PROMPT_BASELINE = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

Get Train Data

In [3]:
conv_train = get_train_data()[18]
print(conv_train)

P01: 너 스키장 가본 적 있어?/nP02: 아니 ㅠㅠ 가보고 싶은데 기회가 안돼서 가본 적이 없어/nP02: 너는?/nP01: 나도 없어/nP01: 근데 이번 겨울엔 꼭 가기로 하긴 했지만 ㅜㅜ/nP01: 근데 좀 부끄러울듯 키키/nP02: 너는 뭔가 잘 탈 것 같은데?/nP02: 왜 부끄러워?/nP01: 잘 못타고 계속 넘어질까봐 키키/nP01: 금잔디 *** 같이 탈까봐/nP02: 그건 로보트 아니야?/nP02: ***보단 잘 탈듯/nP01: ㅇㅈ 난 보드는 발에 족쇄찬 느낌일거 같고 스키 배워볼거야/nP01: 넌?/nP02: 근자감인가?/nP02: 나는 잘 탈 것 같은데?/nP01: ^^ 대단한걸~?/nP01: 근데 수영처럼 에너지 소모 커서 컵라면 꿀맛일듯/nP02: 너도 알구나?/nP02: 수영 후 먹는 컵라면이 세상에서 제일 맛있는듯/nP01: 맞어 그리고 바다에서 놀고 나서 먹는 뽕따 진리지/nP02: 뽕따? 그것도 국룰이야?/nP02: 처음 알았네 키키/nP01: 키키 나만의 룰이였나봐/nP01: 근데 나 어릴 때 어린이스포츠 수영 다녀서 살빼려 했는데 더 먹어서 더 쪄썽/nP02: 키키 수영 에너지 소모가 크니까 더 먹게 되는 것 같아 키키/nP02: 나도 그랬음 키키/nP02: 나는 수영 끝나면 맨날 그 옆 매점에서 제일 저렴한 도시락 컵라면 먹은 기억 있어/nP01: 키키 도시락 추억이다/nP01: 난 러시아에서 그거에 마요네즈 뿌려먹는다길래 궁금해서 대학생 때 첨 먹어봄/nP02: 예? 도시락에 마요네즈요?/nP02: 맛 어땠어? 이상할 것 같은데


Few-Shot

In [4]:
summary_sample = """P01과 P02는 스키장 경험이 없지만, P01은 이번 겨울 스키 도전을 계획 중이며 넘어질까 걱정한다.
수영 후 컵라면과 바다에서 뽕따 먹는 추억에 공감하며 어릴 때의 운동과 식습관 이야기를 나눈다.
P01은 대학생 때 러시아 스타일로 도시락에 마요네즈를 뿌려 먹어본 독특한 경험을 공유한다."""

prompt = f"""당신은 요약 전문가입니다. 사용자 대화들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 사용자 대화와 요약 예시입니다.
예시 대화:
{conv_train}
예시 요약 결과:
{summary_sample}
    
아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [6]:
models = [
    'claude-3-haiku-20240307',
    'gemini-1.5-flash',
    'gpt-3.5-turbo-0125'
]

scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(5)):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
for model in scores:
    print(scores[model], model)

[8, 7, 7, 6, 6] claude-3-haiku-20240307
[9, 6, 7, 8, 6] gemini-1.5-flash
[6, 5, 8, 6, 3] gpt-3.5-turbo-0125


Chain-of-thought

In [8]:
summary_sample = """P01과 P02는 스키장 경험이 없지만, P01은 이번 겨울 스키 도전을 계획 중이며 넘어질까 걱정한다.
수영 후 컵라면과 바다에서 뽕따 먹는 추억에 공감하며 어릴 때의 운동과 식습관 이야기를 나눈다.
P01은 대학생 때 러시아 스타일로 도시락에 마요네즈를 뿌려 먹어본 독특한 경험을 공유한다."""

prompt = f"""당신은 요약 전문가입니다. 사용자 대화들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 대화를 요약할 때는 다음 단계를 따라주세요:

1. 대화 참여자 파악: 대화에 참여하는 사람들의 수와 관계를 파악합니다.
2. 주제 식별: 대화의 주요 주제와 부차적인 주제들을 식별합니다.
3. 핵심 내용 추출: 각 주제에 대한 중요한 정보나 의견을 추출합니다.
4. 감정과 태도 분석: 대화 참여자들의 감정이나 태도를 파악합니다.
5. 맥락 이해: 대화의 전반적인 맥락과 배경을 이해합니다.
6. 특이사항 기록: 대화 중 특별히 눈에 띄는 점이나 중요한 사건을 기록합니다.
7. 요약문 작성: 위의 단계에서 얻은 정보를 바탕으로 간결하고 명확한 요약문을 작성합니다.
각 단계를 수행한 후, 최종적으로 전체 대화를 200자 내외로 요약해주세요.

예시 대화:
{conv_train}
예시 요약 결과:
{summary_sample}
    
아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [9]:
models = [
    'claude-3-haiku-20240307',
    'gemini-1.5-flash-001',
    'gpt-3.5-turbo-0125'
]
scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(5)):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
for model in scores:
    print(scores[model], model)

[6, 6, 7, 6, 6] claude-3-haiku-20240307
[6, 4, 6, 8, 9] gemini-1.5-flash-001
[5, 4, 6, 6, 6] gpt-3.5-turbo-0125


In [15]:

conv_train = """P01: 코로나가 좀 잠잠해지면 해외여행 중에 가고 싶은 곳 있어?
P02: 난 호주 한번 다시 다녀오고 싶엉 키키
P02: 아님... 원래 일본도 가보고 싶었는데 무서워서 안 갈래 키키
P01: 일본은 예전 노 재팬 한 이후로 사실 아직도 웬만하면 일본꺼 안 쓰거든 키키
P01: 나도 일본은 별로고 키키
P01: 나는 하와이 괌 이런 곳 가보고 싶어 키키
P02: 휴양지 좋지 키키
P02: 아아! 베트남 다낭 가보고 싶어~
P02: 좋다던데 키키
P01: 맞아 맞아 나 남편이랑 연애 초기에 남편 첫 직장 입사하고 그 해에 바로 거기로 해외 워크샵 갔는데 진짜 완전 질투나서 키키 연락하다 싸움 키키
P02: 키키 질투 났어? 키키
P02: 아 웃곀 키키
P02: 나중에 우리끼리 가자
P01: 키키 아니 사귄 지 얼마 안 됐는데 아 얼마 안된 건 아닌가 키키
P01: 무튼 연락도 안 되고 전화하면 혼자 신나 있고 난 연락 기다리느라 계속 신경은 거기로만 가있는대 얘는 아니니까 키키
P01: 흔한 그 연인들의 싸움이었지 키키
P01: 무튼 가보고 싶긴 해
P02: 키키 신나 있었어?
P02: 이런 키키
P02: 진짜 떠나고 싶당~
P01: 웅 키키 중딩 때 태권도에서 일본 간 거 말고 첫 해외여행이라고 매우 들떠있었지 키키
P01: 호텔도 엄청 좋은 데로 가고 키키
P01: 첫 직장에 첫 해외니 얼마나 좋았겠어 키키
P02: 엄청 좋지 키키
P02: 우리 대학교 졸업 여행 간 거 기억나?
P02: 태국 키키
P02: 난 그게 첫 해외여행이었어
P01: 기억나지 키키
P01: 내 얼굴에서만 플래쉬 터지고... 키키
P01: 아 재밌었지 웃기고 키키
P01: 태국 가보고 그다음에 간 게 호주 워홀이었던 건가?
P02: 키키 아 맞아~ 플래쉬 키키
P02: 옹옹~ 그 담이 호주였지~
P02: 대담해써 아주키키
P01: 그니까 용기 있었어 키키
P01: 대단해
P01: 돈 열심히 모아서 진짜 스위스 꼭 가봐
P01: 죽어도 여한 없을 것 같은 그런 느낌이야... 키키
P02: 신혼으로 스위스 갔지?
P02: 그때 사진 보고 넘 부러웠징~ 키키
P02: 여행 갈 나라가 너무 많다
P01: 그치 키키 가보고 싶은 나라가 많아
P01: 근데 사실 나는 아시아권 보단 유럽 가고 싶어 ㅠ
P02: 유럽은 뭔가 고급 지고 멋진 느낌이야 키키
P02: 죽기 전에는 가보겠지?
P01: 그치 키키 그냥 여유로운 느낌이 좋아
P01: 근데 뭐 유럽이라고 다 그렇지는 않더라 키키
P01: 좋은 사람도 있지만 나쁘고 위험한 사람도 많고 ㅠㅠ
P02: 맞아~ 위험하고 무서운 사람들 있지 ㅠㅠ
P02: 인도는 여자들 여행 가면 큰일 난대~ 없어져도 모른대~
P01: 그래 맞아 ㅠㅠ
P01: 내 친구는 유럽 어디더라 친구 두 명이랑 같이 여행 갔는데 짐을 통제로 소매치기 당해서 엄청 곤란한 상황이었다고 하더라고 ㅠ
P02: 어후...
P02: 여권도 잃어버렸을 거 아냐 ㅠㅠ
P02: 진짜 멘붕 됐겠다... 으악
P01: 엉 그니까 ㅠ 그래도 뭐 어찌어찌 위기는 넘긴 거 같더라 ㅠ
P01: 해외 여행 가면 가이드 안 끼고 가면 영어 실력 엄청 좋은 사람이랑 가는 거 아닐 땐 소통 문제도 좀 곤란할 때가 있는 거 같아
P02: 맞아~ 영어는 진짜... 잘하고 싶다 키키
P02: 왜 영어 공부를 열심히 안 했을까... 후회 키키"""

prompt = f"""당신은 요약 전문가입니다. 사용자 대화들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 대화를 요약할 때는 다음 단계를 따라주세요:

1. 대화 참여자 파악: 대화에 참여하는 사람들의 수와 관계를 파악합니다.
2. 주제 식별: 대화의 주요 주제와 부차적인 주제들을 식별합니다.
3. 핵심 내용 추출: 각 주제에 대한 중요한 정보나 의견을 추출합니다.
4. 감정과 태도 분석: 대화 참여자들의 감정이나 태도를 파악합니다.
5. 맥락 이해: 대화의 전반적인 맥락과 배경을 이해합니다.
6. 특이사항 기록: 대화 중 특별히 눈에 띄는 점이나 중요한 사건을 기록합니다.
7. 요약문 작성: 위의 단계에서 얻은 정보를 바탕으로 간결하고 명확한 요약문을 작성합니다.
각 단계를 수행한 후, 최종적으로 전체 대화를 200자 내외로 요약해주세요.

아래는 예시 대화와 예시 요약 과정 및 결과 입니다.

예시 대화:
{conv_train}

예시 요약 과정
1. "우리 대학교 졸업 여행 간 거 기억나?"라는 언급과 전반적으로 친밀한 대화 톤을 사용하고 있는 것을 보았을 떄 두 사용자는 오랜 친구 사이로 보입니다.
대화의 시작 부분에서 "코로나가 좀 잠잠해지면 해외여행 중에 가고 싶은 곳 있어?"라고 묻고 있는 것을 보았을 때 코로나 이후 가고 싶은 해외 여행지에 대해 논의하고 있습니다.
따라서 다음과 같이 요약 할 수 있습니다:
최소 대학 생활부터 함께 한 매우 친밀한 사이의 두 사용자가 코로나가 잠잠해졌을 때 방문하고 싶은 해외 여행지에 대해 일상적이고 가벼운 톤으로 대화하고 있습니다.

2. 대화 중 호주, 일본, 하와이, 괌, 베트남 다낭, 스위스, 유럽들이 언급하고 있습니다.
남편의 첫 직장 워크샵, 대학교 졸업 여행, 호주 워킹홀리데이 등의 경험을 이야기하면서 과거 여행 경험을 공유하며 추억을 회상하고 있습니다.
따라서 다음과 같이 요약 할 수 있습니다:
여행지로는 하와이, 괌, 스위스, 호주, 베트남 다낭 등을 언급하며 남편과의 연락 관련 다툼이나 졸업여행 관련 추억을 회상합니다.

3. 소매치기, 여권 분실, 인도에서의 여성 여행자 위험 등을 언급하며 해외 여행의 위험성에 대해 우려를 표현하고 있습니다.
"해외 여행 가면 가이드 안 끼고 가면 영어 실력 엄청 좋은 사람이랑 가는 거 아닐 땐 소통 문제도 좀 곤란할 때가 있는 거 같아"라는 언급과 "왜 영어 공부를 열심히 안 했을까... 후회"라는 표현이 있는 것을 보았을 때 언어 장벽의 어려움을 인식하고 영어 실력 향상에 대한 욕구를 표현합니다.
따라서 다음과 같이 요약 할 수 있습니다:
또한 여행 중 발생하는 위험에 대한 우려도 표하고 있으며, 해외여행 시 언어 장벽의 어려움을 인식하고 영어 실력을 향상시키고 싶다는 마음을 가볍게 표현합니다.

예시 요약 결과
최소 대학 생활부터 함께 한 매우 친밀한 사이의 두 사용자가 코로나가 잠잠해졌을 때 방문하고 싶은 해외 여행지에 대해 일상적이고 가벼운 톤으로 대화하고 있습니다.
여행지로는 하와이, 괌, 스위스, 호주, 베트남 다낭 등을 언급하며 남편과의 연락 관련 다툼이나 졸업여행 관련 추억을 회상합니다.
또한 여행 중 발생하는 위험에 대한 우려도 표하고 있으며, 해외여행 시 언어 장벽의 어려움을 인식하고 영어 실력을 향상시키고 싶다는 마음을 가볍게 표현합니다.
    
아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

In [16]:
models = [
    'claude-3-haiku-20240307',
    'gemini-1.5-flash-001',
    'gpt-3.5-turbo-0125'
]
scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(5)):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [22]:
for model in scores:
    print(scores[model], model)

[8, 9, 9, 9, 6, 6, 4, 6, 6, 7, 6, 4, 4, 7, 6, 6, 6, 6, 6, 8, 6, 9, 6, 9, 8, 8, 5, 6, 6, 6, 5, 6, 7, 7, 4, 7, 9, 9, 2, 8] claude-3-haiku-20240307


In [26]:
models = [
    'claude-3-haiku-20240307',
    'gemini-1.5-flash',
    'gpt-3.5-turbo-0125'
]
scores = {model: [] for model in models}
pattern = r'\[\[\d+\]\]'

for model in models:
    for i in tqdm(range(len(get_eval_data()))):
        summary = summarize(
            conversation=get_eval_data()[i],
            prompt=prompt,
            model=model
        )
        eval_comment = pointwise_eval(get_eval_data()[i], summary)
        match = re.search(pattern, eval_comment)
        matched_string = match.group(0)
        score = int(matched_string[2])
        scores[model].append(score)

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [27]:

for model in scores:
    print(scores[model], model)

[8, 9, 9, 9, 6, 5, 5, 6, 6, 9, 6, 6, 4, 8, 6, 6, 5, 5, 6, 9, 6, 9, 6, 9, 8, 7, 6, 6, 6, 6, 6, 6, 7, 7, 6, 7, 9, 9, 2, 7] claude-3-haiku-20240307
[7, 7, 8, 8, 6, 8, 6, 6, 6, 9, 7, 9, 4, 6, 7, 7, 6, 8, 6, 7, 7, 9, 7, 8, 7, 6, 6, 9, 6, 6, 9, 7, 6, 8, 9, 6, 6, 6, 8, 8] gemini-1.5-flash
[7, 6, 7, 6, 4, 4, 4, 4, 6, 6, 5, 5, 4, 4, 5, 6, 4, 6, 5, 6, 5, 4, 6, 6, 8, 6, 4, 4, 6, 6, 6, 6, 6, 6, 7, 4, 6, 4, 7, 5] gpt-3.5-turbo-0125


In [28]:
import math

for model in scores:
    mean = sum(scores[model]) / len(scores[model])
    variance = sum((x - mean) ** 2 for x in scores[model]) / (len(scores[model]) - 1)
    std_dev = math.sqrt(variance)
    print(f'{model}: {mean} / {round(std_dev, 2)}')

claude-3-haiku-20240307: 6.7 / 1.62
gemini-1.5-flash: 7.05 / 1.2
gpt-3.5-turbo-0125: 5.4 / 1.1


비교

In [29]:
PROMPT_BASELINE = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:"""

summary = summarize(
    conversation=get_eval_data()[10],
    prompt=PROMPT_BASELINE,
    model='claude-3-haiku-20240307'
)
print(summary)

1. P01은 #@이름#에게 새해 인사를 보냈고, 결혼식 초대를 받았는지 궁금해했다.
2. P02는 #@이름#과 친구가 아니며, 결혼식에 가지 않을 것이라고 말했다.
3. P01과 P02는 #@이름#의 행동에 대해 궁금해하며, 임신 여부와 직업 등을 추측하는 대화를 나누었다.


In [30]:
summary = summarize(
    conversation=get_eval_data()[10],
    prompt=prompt,
    model='claude-3-haiku-20240307'
)
print(summary)

두 사용자가 서로 알고 있는 지인의 결혼 소식에 대해 농담 섞인 대화를 나누고 있습니다. 지인이 갑자기 새해 인사를 보냈다는 것에 대해 의아해하며, 지인이 결혼식에 초대하지 않을 것이라고 예상하고 있습니다. 또한 지인의 결혼 동기에 대해 추측하며 농담을 주고받고 있습니다.
